# Módulo 3: Entrenamiento y Testeo de un modelo de análisis de Sentimiento

En esta etapa nos dedicaremos a entrenar y probar un modelo de anaálisis de sentimiento. Aquí entrenaremos dos modelos distintos de Machine Learning y veremos, mediante diferentes métricas, cual de ellos arroja un mejor resultado.

In [1]:
# En primer lugar, hacemos los imports necesarios
import pandas as pd

# Luego importamos el dfcorpus preprocesado de la etapa anterior
dfcorpus = pd.read_csv('./MusicalInstrumentsPreprocesado.csv')

# Nos aseguramos que el importe se ha realizado correctamente
dfcorpus.head()

,overall,reviewText
0,1,good beginner
1,1,recommend starter ukulele kit ha everything le...
2,1,gdaughter received christmas present last year...
3,0,according order history bought kit towards end...
4,0,please pay attention better review poor qualit...


Ahora aseguremonos por última vez que nuestro DataFrame no presenta datos vacíos.

In [2]:
dfcorpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231344 entries, 0 to 231343
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   overall     231344 non-null  int64 
 1   reviewText  230480 non-null  object
dtypes: int64(1), object(1)
memory usage: 3.5+ MB


Podemos observar que tenemos algunos valores nulos producto del preprocesado. Vamos a eliminar estos registros antes de realizar el split. Podemos hacerlo sin mayor problema porque son pocos en comparación a la cantidad de información que poseemos.

In [3]:
# Eliminamos los registros donde el campo reviewText se encuentra vacío
dfcorpus.dropna(inplace=True)

# Nos aseguramos que el proceso ha ocurrido correctamente
dfcorpus.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 230480 entries, 0 to 231343
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   overall     230480 non-null  int64 
 1   reviewText  230480 non-null  object
dtypes: int64(1), object(1)
memory usage: 5.3+ MB


Ahora realizaremos la separación en conjunto de entrenamiento y conjunto de prueba.

In [4]:
# Hacemos los imports necesarios para la separación
from sklearn.model_selection import train_test_split

# Hacemos la separación en entrenamiento y prueba para la variable predictora y la variable a predecir (test_size = 10%)
xtrain, xtest, ytrain, ytest = train_test_split(dfcorpus['reviewText'], dfcorpus['overall'], test_size=0.1, random_state=42, shuffle=True)

# Ahora veremos las dimensiones de nuestras variables
print(f'Shape xtrain: {xtrain.shape}')
print(f'Shape ytrain: {ytrain.shape}')
print(f'Shape xtest: {xtest.shape}')
print(f'Shape ytest: {ytest.shape}')

Shape xtrain: (207432,)
Shape ytrain: (207432,)
Shape xtest: (23048,)
Shape ytest: (23048,)


A continuación haremos una extracción de características mediante la implementación de codificación basado en Bolsa de Palabras (BoW). Para eso utilizaremos la clase de Scikit-Learn CountVectorizer.

In [5]:
# En primer lugar, hacemos los imports necesarios
from sklearn.feature_extraction.text import CountVectorizer

# Ahora creamos el objeto
cv = CountVectorizer(max_features=2000, ngram_range=(1,2), max_df=0.95, min_df=5)

# Ajustamos y transformamos a nuestra variable predictora de entrenamiento
xtrain_cv = cv.fit_transform(xtrain)

# Utilizamos el ajuste realizado con las variables predictoras de entrenamiento para transformar las variables predictoras de prueba
xtest_cv = cv.transform(xtest)

En este momento aplicaremos un modelo de Regresión Logística y veremos que tan buenos resultados nos trae.

In [14]:
# En primer lugar, hacemos los imports necesarios
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Ahora creamos la lista de posibles hiperparámetros con el fin de decidir el óptimo
possibleMaxIter = [500, 1000, 2000, 5000]

# Luego construimos y ajustamos el GridSearchCV
param_grid = {'max_iter' : possibleMaxIter}
grid = GridSearchCV(LogisticRegression(random_state=42), param_grid=param_grid, cv=10, verbose=2)
grid.fit(xtrain_cv, ytrain)

# Posteriormente inicializamos los hiperparámetros óptimos
maxIterOptimo = grid.best_params_['max_iter']

# Ahora inicializamos el mismo modelo nuevamente, pero esta vez con los parámetros óptimos
lr = LogisticRegression(random_state=42, max_iter=maxIterOptimo)
lr.fit(xtrain_cv, ytrain)

Fitting 10 folds for each of 4 candidates, totalling 40 fits
[CV] END .......................................max_iter=500; total time=   9.0s
[CV] END .......................................max_iter=500; total time=   4.7s
[CV] END .......................................max_iter=500; total time=   7.0s
[CV] END .......................................max_iter=500; total time=   6.1s
[CV] END .......................................max_iter=500; total time=   6.2s
[CV] END .......................................max_iter=500; total time=   6.3s
[CV] END .......................................max_iter=500; total time=   7.0s
[CV] END .......................................max_iter=500; total time=   7.1s
[CV] END .......................................max_iter=500; total time=   7.5s
[CV] END .......................................max_iter=500; total time=   5.8s
[CV] END ......................................max_iter=1000; total time=   6.9s
[CV] END ......................................m

LogisticRegression(max_iter=500, random_state=42)

In [15]:
# -------------------------------- HACEMOS LAS PREDICCIONES --------------------------------

# Hacemos las predicciones a ytrain y ytest
lr_ytrain_pred = lr.predict(xtrain_cv)
lr_ytest_pred = lr.predict(xtest_cv)

# -------------------------------- CALCULAMOS LAS MÉTRICAS --------------------------------

# Calculamos el accuracy
print('Accuracy')
print(f'Train: {accuracy_score(ytrain, lr_ytrain_pred)}')
print(f'Test: {accuracy_score(ytest, lr_ytest_pred)} \n')

# Calculamos la precision
print('Precision')
print(f'Train: {precision_score(ytrain, lr_ytrain_pred)}')
print(f'Test: {precision_score(ytest, lr_ytest_pred)} \n')

# Calculamos el recall
print('Recall')
print(f'Train: {recall_score(ytrain, lr_ytrain_pred)}')
print(f'Test: {recall_score(ytest, lr_ytest_pred)} \n')

# Calculamos el f1_score
print('F1 Score')
print(f'Train: {f1_score(ytrain, lr_ytrain_pred)}')
print(f'Test: {f1_score(ytest, lr_ytest_pred)} \n')

Accuracy
Train: 0.7914208029619345
Test: 0.7870531065602221 

Precision
Train: 0.7962641434680774
Test: 0.7938397285304098 

Recall
Train: 0.9427582020363676
Test: 0.9406742963192082 

F1 Score
Train: 0.8633408928672954
Test: 0.8610419026047565 



Podemos observar que obtenemos resultados bastantes decentes y sin overfitting aplicando una Regresión Logística sencilla. Esto puede deberse a que tenemos suficientes datos para entrenar cómodamente al modelo y al hecho de que se hizo un buen preprocesado de texto...

Ahora veremos que tan buenos resultados podemos obtrener con un modelo de Clasificación Binaria con Bosques Aleatorios.

In [16]:
# En primer lugar, hacemos los imports necesarios
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Ahora creamos la lista de posibles hiperparámetros con el fin de decidir el óptimo
possibleMaxDepth = range(1,6)

# Luego construimos y ajustamos el GridSearchCV
param_grid = {'max_depth' : possibleMaxDepth}
grid = GridSearchCV(RandomForestClassifier(random_state=42), param_grid=param_grid, cv=10, verbose=2)
grid.fit(xtrain_cv, ytrain)

# Posteriormente inicializamos los hiperparámetros óptimos
maxDepthOptimo = grid.best_params_['max_depth']

# Ahora inicializamos el mismo modelo nuevamente, pero esta vez con los parámetros óptimos
rfc = RandomForestClassifier(random_state=42, max_depth=maxDepthOptimo)
rfc.fit(xtrain_cv, ytrain)

Fitting 10 folds for each of 5 candidates, totalling 50 fits
[CV] END ........................................max_depth=1; total time=   3.4s
[CV] END ........................................max_depth=1; total time=   3.3s
[CV] END ........................................max_depth=1; total time=   4.2s
[CV] END ........................................max_depth=1; total time=   3.3s
[CV] END ........................................max_depth=1; total time=   3.3s
[CV] END ........................................max_depth=1; total time=   3.8s
[CV] END ........................................max_depth=1; total time=   3.4s
[CV] END ........................................max_depth=1; total time=   3.3s
[CV] END ........................................max_depth=1; total time=   3.2s
[CV] END ........................................max_depth=1; total time=   4.1s
[CV] END ........................................max_depth=2; total time=   3.7s
[CV] END .......................................

RandomForestClassifier(max_depth=5, random_state=42)

In [19]:
# -------------------------------- HACEMOS LAS PREDICCIONES --------------------------------

# Hacemos las predicciones a ytrain y ytest
rfc_ytrain_pred = rfc.predict(xtrain_cv)
rfc_ytest_pred = rfc.predict(xtest_cv)

# -------------------------------- CALCULAMOS LAS MÉTRICAS --------------------------------

# Calculamos el accuracy
print('Accuracy')
print(f'Train: {accuracy_score(ytrain, rfc_ytrain_pred)}')
print(f'Test: {accuracy_score(ytest, rfc_ytest_pred)} \n')

# Calculamos la precision
print('Precision')
print(f'Train: {precision_score(ytrain, rfc_ytrain_pred)}')
print(f'Test: {precision_score(ytest, rfc_ytest_pred)} \n')

# Calculamos el recall
print('Recall')
print(f'Train: {recall_score(ytrain, rfc_ytrain_pred)}')
print(f'Test: {recall_score(ytest, rfc_ytest_pred)} \n')

# Calculamos el f1_score
print('F1 Score')
print(f'Train: {f1_score(ytrain, rfc_ytrain_pred)}')
print(f'Test: {f1_score(ytest, rfc_ytest_pred)} \n')

Accuracy
Train: 0.699511165104709
Test: 0.7019698021520305 

Precision
Train: 0.6993202661044832
Test: 0.7018061827023272 

Recall
Train: 0.9999724069424133
Test: 0.9999381379523662 

F1 Score
Train: 0.8230494732449305
Test: 0.8247569967089318 



Podemos observar que obtenemos resultados bastantes decentes y sin overfitting aplicando una Clasificación con Bosques Aleatorios sencilla. Esto puede deberse a que tenemos suficientes datos para entrenar cómodamente al modelo y al hecho de que se hizo un buen preprocesado de texto...

### Conclusiones del Modelado

En esta etapa probamos dos modelos de Machine Learning, siendo estos Regresión Logística y Clasificación con Bosques Aleatorios. Si bien es cierto que ambos modelos arrojaron resultados bastante decentes, pudimos notar que la Regresión Logística arrojó resultados ligeramente mejores que la Clasificación con Bosques Aleatorios. Rescataremos las métricas calculadas y estudiaremos sus resultados en la etapa posterior.

In [21]:
# Creamos un diccionario con las métricas calculadas de los modelos
metrics_dict = {'Model':['LogisticRegression','LogisticRegression','RandomForestClassifier','RandomForestClassifier'],
                'Category':['Train','Test','Train','Test'],
                'Accuracy':[accuracy_score(ytrain, lr_ytrain_pred), accuracy_score(ytest, lr_ytest_pred), accuracy_score(ytrain, rfc_ytrain_pred), accuracy_score(ytest, rfc_ytest_pred)],
                'Precision':[precision_score(ytrain, lr_ytrain_pred), precision_score(ytest, lr_ytest_pred), precision_score(ytrain, rfc_ytrain_pred), precision_score(ytest, rfc_ytest_pred)],
                'Recall':[recall_score(ytrain, lr_ytrain_pred), recall_score(ytest, lr_ytest_pred), recall_score(ytrain, rfc_ytrain_pred), recall_score(ytest, rfc_ytest_pred)],
                'F1 Score':[f1_score(ytrain, lr_ytrain_pred), f1_score(ytest, lr_ytest_pred), f1_score(ytrain, rfc_ytrain_pred), f1_score(ytest, rfc_ytest_pred)]}

# Ahora creamos un DataFrame a partir del diccionario creado anteriormente
dfmetrics = pd.DataFrame(metrics_dict)

# Ahora nos aseguramos que todo se generó de la manera esperada
dfmetrics.head()

# Guardo el DataFrame en un archivo csv para poder utilizarlo en partes posteriores
dfmetrics.to_csv('MusicalInstrumentsMetrics.csv', index=False)